# AI 流程

本 AI 利用神经网络模型评估玩家“出牌后”的局面。

玩家在出牌时可能有多种选择，每一种可能的出牌对应一种出牌后的局面，AI对所有局面进行评估，从而选择出对己方最有利的一种出牌。

* 根据出牌人的位置选择模型。本 AI 使用三个模型，分别对应地主，地主下家，地主上家。
* 取玩家所有可能的出牌，对出牌后的局面进行编码。
* 将局面数据输入模型，输出对局面的评估，即“地主的胜率”。
* 根据模型输出选择出牌。对于地主来说，选择评分最高的一项，对于农民，则是选择评分最低的一项。

# 模型说明

In [1]:
import numpy as np
import torch
import bot_torch_ln

In [10]:
model = bot_torch_ln.Model()  ## 创建模型
model.eval() ## 使模型工作在 eval 模式（对应的是 train 模式，用于训练模型，两者有所区别）
cp = torch.load("model_tree2/cp77795.pt")  ## 读取模型参数
model.load_state_dict(cp["models_state_dict"][0])  ## 加载模型参数


<All keys matched successfully>

In [11]:
[k for k in cp]

['models_state_dict', 'optizimers_state_dict']

cp 是字典数据，有两项 models_state_dict 是模型参数，用于出牌, optizimers_state_dict 是优化器参数，用于训练。

models_state_dict 是有三项数据的 tuple, 每一项代表一个模型的参数。本 AI 对每个位置（地主，地主上家，地主下家）分别使用不同的模型。

## 模型调用

In [12]:
x = torch.randn((1,15,3,15))
with torch.no_grad():  ##固定写法，告诉模型不要计算“梯度”。梯度在训练模型时会用到，应用模型时不需要。
    y = model(x)
print(y)

tensor([[1.2052e-05]])


这里模型的输入数据是 1\*15\*3\*15的四维数组，第一个1代表样本数量，1个样本对应1个输出(1\*1的二维数组)。下面的例子中样本数量是6，因此有6个输出(6\*1的二维数组)。

In [15]:
x = torch.randn((6,15,3,15))
with torch.no_grad():  ##固定写法，告诉模型不要计算“梯度”。梯度在训练模型时会用到，应用模型时不需要。
    y = model(x)
print(y)

tensor([[7.1819e-05],
        [6.1910e-05],
        [8.2589e-02],
        [1.6079e-05],
        [4.1479e-04],
        [1.2750e-05]])


输入数据的编码见 bot_torch_ln.py 中的 getdata 函数的注释